In [1]:
import numpy as np
from scipy.optimize import fsolve
import matplotlib.pyplot as plt
from IPython.display import display, clear_output, Audio
%matplotlib inline
plt.rcParams['figure.dpi']= 150

First, we define parameters:

In [2]:
# Transport parameters
s_perm = 0.1
u_perm = 0.1
w_perm = 0.1
S_input = 0.1
U_input = 1
C_osm = 2

# discretization
N = 10
dx = 1/N

Now, we define the difference quotient operator

In [3]:
def diff_x(y): 
    n = len(y)-1
    diff = np.zeros(n)
    for l in range(n):
        diff[l] = (y[l+1]-y[l])/dx
    
    return diff

def avg(y):
    n = len(y)-1
    average = np.zeros(n)
    for l in range(n):
        average[l] = (y[l+1]+y[l])/2
    
    return average

Next, we define $u_0$, $s_0$:

In [4]:
def su_0(s_avg,q_avg,u_avg,s_1,q_out,u_out):
    u_0 = np.zeros(N)
    s_0 = np.zeros(N)
    R = np.zeros(N)
    
    # R[:] = (u[:-1]*q[:-1] - u[-1]*q[-1])/(S_input*(1-(s[:-1]/s[-1])))
    # R[:] = (u_avg*q_avg - u[-1]*q[-1])/(S_input*(1-(s_avg/s[-1])))
    R[:] = (u_avg*q_avg - u_out)/(S_input*(1-(s_avg/s_1)))
    # s_0[:] = S_input*(1-(s[:-1]/s[-1])-(2/(2+R[:])))/(q[:-1]-q[-1]-(S_input/s[-1]))
    # s_0[:] = S_input*(1-(s_avg/s[-1])-(2/(2+R[:])))/(q_avg-q[-1]-(S_input/s[-1]))
    s_0[:] = S_input*(1-(s_avg/s_1)-(2/(2+R[:])))/(q_avg-q_out-(S_input/s_1))
    u_0[:] = R[:]*s_0[:]

    return s_0,u_0

In [5]:
def res(y):
    s = y[:(N+1)].copy()
    # s = np.abs(y[:(N+1)])
    q = y[(N+1):2*(N+1)].copy()
    # q = np.abs(y[(N+1):2*(N+1)])
    u = y[2*(N+1):3*(N+1)].copy()
    # u = np.abs(y[2*(N+1):3*(N+1)])
    s_1 = y[3*(N+1):4*(N+1)].copy()
    q_out = y[4*(N+1):5*(N+1)].copy()
    u_out = y[5*(N+1):6*(N+1)].copy()
    s_avg = avg(s)
    q_avg = avg(q)
    u_avg = avg(u)
    s_1_avg = avg(s_1)
    q_out_avg = avg(q_out)
    u_out_avg = avg(u_out)
    s_0,u_0 = su_0(s_avg,q_avg,u_avg,s_1_avg,q_out_avg,u_out_avg)
    

    # residue = np.zeros(3*(N+1))
    residue = np.zeros(6*(N+1))
    res_s = residue[:(N+1)]
    res_q = residue[(N+1):2*(N+1)]
    res_u = residue[2*(N+1):3*(N+1)]
    res_s_1 = residue[3*(N+1):4*(N+1)]
    res_q_out = residue[4*(N+1):5*(N+1)]
    res_u_out = residue[5*(N+1):6*(N+1)]

    # R_0 = (u[0]*q[0] - u[-1]*q[-1])/(S_input*(1-(s[0]/s[-1])))
    R_0 = (u[0]*q[0] - u_out[0])/(S_input*(1-(s[0]/s_1[0])))
    # s_00 = S_input*(1-(s[0]/s[-1])-(2/(2+R_0)))/(q[0]-q[-1]-(S_input/s[-1]))
    s_00 = S_input*(1-(s[0]/s_1[0])-(2/(2+R_0)))/(q[0]-q_out[0]-(S_input/s_1[0]))
    # res_s[0] = s[0] - s_00 - s[-1]*(1 - (s_00/S_input)*(((2*S_input+U_input)/C_osm) - q[-1]))
    res_s[0] = s[0] - s_00 - s_1[0]*(1 - (s_00/S_input)*(((2*S_input+U_input)/C_osm) - q_out[0]))
    # res_s[1:] = diff_x(s) - (s_perm*s[-1]/S_input)*(s_avg - s_0)
    res_s[1:] = diff_x(s) - (s_perm*s_1_avg/S_input)*(s_avg - s_0)
    res_q[0] = q[0] - U_input/C_osm
    res_q[1:] = diff_x(q) + w_perm*(2*s_0 + u_0 - u_avg)
    res_u[0] = u[0] - C_osm
    res_u[1:] = diff_x(u) - (1/q_avg)*(
        w_perm*u_avg*(2*s_0+u_0 - u_avg)
        - u_perm*(u_avg - u_0)
    )
    res_s_1[0] = s[-1] - s_1[-1]
    res_s_1[1:] = diff_x(s_1)
    res_q_out[0] = q[-1] - q_out[-1]
    res_q_out[1:] = diff_x(q_out)
    res_u_out[0] = u[-1]*q[-1] - u_out[-1]
    res_u_out[1:] = diff_x(u_out)

    return residue

Finally, we put everything together:

In [10]:
# Transport parameters
s_perm = 1
u_perm = 1
w_perm = 1
S_input = .001
U_input = 2
C_osm = 2

# discretization
N = 4
dx = 1/N

#Initial guess
# y_init = np.zeros(3*(N+1))
y_init = np.zeros(6*(N+1))
s_init = y_init[:(N+1)]
q_init = y_init[(N+1):2*(N+1)]
u_init = y_init[2*(N+1):3*(N+1)]
s_1_init = y_init[3*(N+1):4*(N+1)]
q_out_init = y_init[4*(N+1):5*(N+1)]
u_out_init = y_init[5*(N+1):6*(N+1)]
for n in range(N+1):
    # s_init[n] = 0.1*n + C_osm/2
    s_init[n] = n + C_osm/2
    u_init[n] = 2*n + C_osm
    q_init[n] = (1- n/(2*N))*U_input/C_osm
s_1_init[:] = s_init[-1].copy()
q_out_init[:] = q_init[-1].copy()
u_out_init[:] = u_init[-1]*q_init[-1]

y = fsolve(res,y_init)
s = y[:(N+1)]
q = y[(N+1):2*(N+1)]
u = y[2*(N+1):3*(N+1)]
s_1 = y[3*(N+1):4*(N+1)]
q_out = y[4*(N+1):5*(N+1)]
u_out = y[5*(N+1):6*(N+1)]
s_0,u_0 = su_0(avg(s),avg(q),avg(u),avg(s_1),avg(q_out),avg(u_out))
osm = 2*s_0+u_0